In [ ]:
!git clone https://github.com/threestudio-project/threestudio.git

In [ ]:
!sudo apt-get update
!sudo apt-get install mesa-common-dev libegl1-mesa-dev -y
!sudo apt-get install ninja-build  -y

In [ ]:
%pip install --upgrade pip

In [ ]:
%cd /notebooks/threestudio

In [ ]:
!git pull

In [ ]:
""" %pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 --extra-index-url https://download.pytorch.org/whl/cu113 """
%pip install ninja
%pip install  torch==2.0.0+cu118 torchvision==0.15.1+cu118 --index-url https://download.pytorch.org/whl/cu118 
%pip install mediapipe


In [ ]:
%pip install  -r requirements.txt   # --force-reinstall  --upgrade

In [ ]:
%pip install lightning==2.0.0 omegaconf==2.3.0 jaxtyping typeguard diffusers transformers accelerate opencv-python tensorboard matplotlib imageio imageio[ffmpeg] trimesh bitsandbytes sentencepiece safetensors huggingface_hub libigl xatlas 
%pip install open3d plotly # mesh visualization

In [ ]:
%pip install git+https://github.com/KAIR-BAIR/nerfacc.git@v0.5.2
%pip install git+https://github.com/NVlabs/nvdiffrast.git

In [ ]:
%pip install git+https://github.com/NVlabs/tiny-cuda-nn/#subdirectory=bindings/torch

In [ ]:
!mkdir load/zero123
%cd load/zero123
!wget https://zero123.cs.columbia.edu/assets/zero123-xl.ckpt
%cd ../../

In [ ]:
!rm -rf ./outputs

In [ ]:
import os
%env CUDA_VISIBLE_DEVICES=0
# Set environment variable
name = "dog_robot_side"
prompt = "an 3D render of a cartoon skeleton of a dog robot, detailed, 8k"
last="last"
%cd /notebooks/threestudio

In [ ]:
%run launch.py --config configs/magic123-coarse-sd.yaml --train --gpu 0 data.image_path=load/images/hamburger_rgba.png system.prompt_processor.prompt="a delicious hamburger"


In [ ]:
""" data.default_elevation_deg=0
data.default_azimuth_deg=90
data.default_camera_distance=1
system.freq.guidance_eval=0 """
cmd = f''' --train --gpu 0
--config configs/zero123_64.yaml
data.image_path="load/images/{name}_rgba.png"
name="{name}"
tag=Phase1
system.loggers.wandb.enable=false
use_timestamp=False
data.default_azimuth_deg=0
data.random_camera.batch_size=14
system.loss.lambda_depth=0.05
system.loss.lambda_depth_rel="[0, 0, 0.05, 100]"
system.loss.lambda_normal="[0, 0, 0.05, 100]"
system.geometry.density_blob_scale=8.
system.freq.guidance_eval=37
system.guidance.cond_elevation_deg=0
'''

%run launch.py {cmd}

In [ ]:
# Phase 1.5 - 512 refine
#system.loggers.wandb.project="zero123" system.loggers.wandb.name=${NAME}_Phase1p5

cmd = f'''--config configs/zero123-geometry.yaml --train --gpu 0 
data.image_path=./load/images/{name}_rgba.png 
system.geometry_convert_from=./outputs/{name}/Phase1/ckpts/{last}.ckpt 
use_timestamp=False name={name} tag=Phase1p5'''

%run launch.py {cmd}

In [ ]:
# Phase 2 - dreamfusion
#data.random_camera.batch_size=2 trainer.accumulate_grad_batches=4 system.freq.guidance_eval=0
# system.freq.guidance_eval=0 system.loggers.wandb.enable=false system.loggers.wandb.project="zero123" system.loggers.wandb.name=${NAME}_Phase2
tag="Phase1p5"
last="last"
cmd = f'''--config configs/experimental/imagecondition_zero123nerf.yaml 
--train --gpu 0 data.image_path=./load/images/{name}_rgba.png
system.prompt_processor.prompt="{prompt}" 
system.weights="./outputs/{name}/{tag}/ckpts/{last}.ckpt" 
system.freq.guidance_eval=23
name={name} tag=Phase2 system.loggers.wandb.enable=false  use_timestamp=False'''

%run launch.py {cmd}

In [ ]:

# Phase 2 - dreamfusion
cmd = f'''--config configs/experimental/my_imagecondition_zero123nerf_refine.yaml 
--train --gpu 0  system.prompt_processor.prompt="{prompt}" 
system.geometry_convert_from="./outputs/{name}/{phase}/ckpts/{last}.ckpt" 
name={name} 
tag=Phase2_refine system.loggers.wandb.enable=false 
use_timestamp=False
data.image_path=./load/images/{name}_rgba.png
'''

%run launch.py {cmd}

In [ ]:
tag="Phase1p5"
last="last"
cmd=f'''--config "outputs/{name}/{tag}/configs/parsed.yaml" 
--export --gpu 0 resume="outputs/{name}/{tag}/ckpts/{last}.ckpt" 
system.exporter.context_type=cuda  system.exporter_type=mesh-exporter'''
%run launch.py {cmd}

TEXT MESH

In [ ]:
# uses DeepFloyd IF, requires ~15GB VRAM
cmd= f'''
--config configs/textmesh-if.yaml --train --gpu 0 tag=DRAGON system.prompt_processor.prompt="{prompt}"
system.prompt_processor_type="stable-diffusion-prompt-processor"
system.guidance_type="stable-diffusion-guidance"
system.guidance.pretrained_model_name_or_path="stabilityai/stable-diffusion-2-1"
system.prompt_processor.pretrained_model_name_or_path="stabilityai/stable-diffusion-2-1"
'''
%run launch.py {cmd}

In [ ]:
tag="DRAGON@20230726-085910"
name="textmesh-if"
cmd=f'--config "outputs/{name}/{tag}/configs/parsed.yaml" --export --gpu 0 resume="outputs/{name}/{tag}/ckpts/last.ckpt" system.exporter.context_type=cuda  system.exporter_type=mesh-exporter'
%run launch.py {cmd}

CONTROL 4D

In [ ]:
cmd = f'''--config configs/control4d-static.yaml --train --gpu 0 
data.dataroot="YOUR_DATAROOT/twindom" 
system.prompt_processor.prompt="{prompt}" 
name={name} 
tag=Phase1 
system.loggers.wandb.enable=false 
use_timestamp=False'''

%run launch.py {cmd}

MAGIC123

In [ ]:
%env CUDA_VISIBLE_DEVICES=0
workspace ="dog_robot_side"
image_path = f'/notebooks/threestudio/load/images/{workspace}_rgba.png'
image_path_without_rgba = image_path.replace('_rgba', '')
prompt = '''
high quality amazing detailed art of 
an 3D stylized dog robot
'''
# If the image contains non-front-facing objects, specifying the approximate elevation 
# and azimuth angle by setting data.default_elevation_deg and data.default_azimuth_deg can be helpful. 
# In threestudio, top is elevation +90 and bottom is elevation -90; left is azimuth -90 and right is azimuth +90.
default_elevation_deg = 0
default_azimuth_deg = -85
%cd /notebooks/threestudio

In [ ]:
%pip install opencv-python==4.8.0.76 numpy==1.23 --force-reinstall --upgrade

In [ ]:
%run launch.py --config configs/magic123-coarse-sd.yaml --train --gpu 0 \
data.image_path="{image_path}" system.prompt_processor.prompt="{prompt}" \
tag=Phase1 data.default_azimuth_deg={default_azimuth_deg} data.default_elevation_deg={default_elevation_deg} \
data.requires_depth=True data.requires_normal=True \
use_timestamp=False \
system.loss.lambda_depth=0.05 system.loss.lambda_depth_rel="[0, 0, 0.05, 100]" system.loss.lambda_normal="[0, 0, 0.05, 100]"


In [ ]:
# Zero123 + Stable Diffusion, ~10GB VRAM
# data.image_path must point to a 4-channel RGBA image
# system.prompt_proessor.prompt must be specified
%run  launch.py --config configs/magic123-refine-sd.yaml --train --gpu 0 data.image_path="{image_path}" \
use_timestamp=False \
system.prompt_processor.prompt="{prompt}" system.geometry_convert_from="outputs/{workspace}/Phase1/ckpts/last.ckpt" tag=Phase2 


In [ ]:
# if you're unsatisfied with the surface extracted using the default threshold (25)
# you can specify a threshold value using `system.geometry_convert_override`
# decrease the value if the extracted surface is incomplete, increase if it is extruded
%run launch.py --config configs/magic123-refine-sd.yaml --train --gpu 0 data.image_path={image_path} \
use_timestamp=False \
system.prompt_processor.prompt={prompt} system.geometry_convert_from="outputs/{workspace}/Phase2/ckpts/last.ckpt"  system.geometry_convert_override.isosurface_threshold=10.